# Scraping du site beerwulf pour enrichir la base de données

- Nous allons scrapper les données relatives aux bières sur le site www.beerwulf.com. L'avantage de ce site est qu'il dispose de données précises sur les bières : des données physico chimiques, mais aussi des indicateurs subjectifs exhaustifs (couleur de la bière, son goût, ou encore sa longueur). Nous utilisons le site https://www.beerwulf.com/fr-be/c/bieres?page=1&container=Bouteille,Canette pour afficher toutes les bières ainsi que les canettes proposées.
- Avant toute chose, il est nécessaire d'explorer la page https://www.beerwulf.com/robots.txt afin de s'assurer que le web-scrapping des données des bières est autorisé. **Ici, il se trouve que c'est le cas.**

On peut alors commencer à importer les librairies et les packages nécessaires au scraping

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd

options = Options()
options.headless = True # On "n'affiche pas" la recherche internet
options.add_argument("--window-size=1920,1200")
DRIVER_PATH = '/Applications/chromedriver'

### Étape 1 : obtenir de l'url de toutes les bières du site de www.berewulf.com
Le catalogue comporte 25 pages de bières. On peut donc automatiser la collecte des urls grâce au package `selenium` puisque les pages sont codées en Javascript. Bs4 ne peut pas aller scraper directement les données recherchées. 

In [2]:
# Boucle qui parcourt toutes les pages du site et va collecter le lien `href` de toutes les bières.
nb_pages = 25
liste_urls = []

for i in range(nb_pages):
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get('https://www.beerwulf.com/fr-be/c?page='+str(i+1)+'&container=Bouteille,Canette&segment=Toutes%20les%20bi%C3%A8res&routeQuery=c')
    elems = driver.find_elements_by_xpath("//div[@id='product-items-container']/a")
    for elem in elems:
        liste_urls.append(elem.get_attribute('href'))

#Vérification
print('Il y a '+str(len(liste_urls))+' urls collectés.')

Il y a 1164 urls collectés.


### Étape 2 : obtenir les caractéristiques de toutes les bières du site, et les stocker dans un dictionnaire
Nous disposons donc de l'url de toutes les bières, donc nous pouvons nous rendre sur ces pages internet grâce à `selenium`. Nous pouvons à présent scraper leurs caractéristiques individuelles.

Pour ce faire, nous avons d'abord besoin de définir des fonctions qui permettront d'aller scraper les caractéristiques des bières. Les données ne sont pas déjà bien rangées dans un tableau, il faut aller les scraper individuellement grâce à leur **"Xpath"**, c'est-à-dire le chemin qui permettra à `selenium` d'accéder à la donnée si ce n'est pas possible avec `BeautifulSoup`.

In [3]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

def valeur_text(xpath):
    """
    Cette fonction scrape des caractéristiques de la bière (degré d'alccolémie, nom de 
    la brasserie, etc) au format text si cette dernière est présente sur la page. 
    """
    try : 
        return driver.find_element_by_xpath(xpath).text
    except (NoSuchElementException, StaleElementReferenceException) as e:
        return float('nan')

def valeur_percent(xpath):
    """
    Cette fonction scrape des caractéristiques de la bière exprimées en pourcentage
    (acidité, amertume, etc) si cette dernière est présente sur la page. 
    """
    try : 
        return driver.find_element_by_xpath(xpath).get_attribute('data-percent')
    except (NoSuchElementException, StaleElementReferenceException) as e:
        return float('nan')


# Scraper l'IBU d'une bière ne peut être entièrement réalisé avec selenium, il faut donc utiliser bs4
def get_ibu(url):
    """
    url : url de la bière selectionnee
    Fonction renvoie l'IBU de la bière grâce à la librairie BeautifulSoup
    """
    request = urllib.request.Request(url, headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    html = urllib.request.urlopen(request)
    html_soup = BeautifulSoup(html, 'html.parser')
    try:
        return html_soup.find_all('strong')[0].text
    except IndexError:
        return float('nan')


On peut désormais parcourir les pages des différentes bières de www.beerwulf.com et stocker leurs caractéristiques dans un dictionnaire.

In [9]:
dico_all_beers = {}

for i in range(len(liste_urls)):
    driver.get(liste_urls[i])

    beer_name = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/h1")
    beer_style = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[1]/a")
    country = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[4]")
    brewery_name = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[5]/a")
    bottle = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[2]")
    abv = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[3]")
    ibu = get_ibu(liste_urls[i])
    intensite = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[1]/dd/div")
    beer_type = valeur_percent("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[1]/a")
    longueur = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[3]/dd/div")
    acidite = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[4]/dd/div")
    amertume = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[5]/dd/div")
    umami = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[6]/dd/div")
    price = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[2]/div/span")
    note = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[1]/span")

    liste_caractéristiques = [beer_name, beer_style, country, brewery_name, bottle, abv, ibu, intensite, longueur, acidite, amertume, umami, price, note]
        
    dico_all_beers[beer_name] = liste_caractéristiques

# Affichage des 5 premières bières scrapées
print(list(dico_all_beers.items())[:5])
print('\nNous avons scrapé '+str(len(dico_all_beers))+' bières.')

[('Brugse Zot blonde', ['Brugse Zot blonde', 'Bière Blonde', 'Belgique', 'Brouwerij De Halve Maan', '33 cl', '6,0%', '23', '60', '40', '20', '40', '0', '€ 2,69', '(3,71)']), ("Brouwerij 't IJ Natte Dubbel", ["Brouwerij 't IJ Natte Dubbel", 'Bière Double', 'Pays-Bas', "Brouwerij 't IJ", '33 cl', '6,5%', '-', '60', '60', '0', '40', '0', '€ 2,65', '(3,44)']), ('Bruut Gajes Tripel', ['Bruut Gajes Tripel', 'Bière Triple', 'Pays-Bas', 'Bruut Bier', '33 cl', '8,0%', '38', '60', '60', '40', '60', '0', '€ 2,99', '(3,76)']), ('Baxbier Koud Vuur', ['Baxbier Koud Vuur', 'Bière Brune', 'Pays-Bas', 'Bax Bier Brouwerij', '33 cl', '6,2%', '38', '80', '40', '20', '60', '0', '€ 3,09', '(3,37)']), ('Bacchus Framboos', ['Bacchus Framboos', 'Bière Fruitée', 'Belgique', 'Brouwerij Van Honsebrouck', '38 cl', '5,0%', '-', '80', '60', '60', '20', '0', '€ 3,49', '(3,60)'])]

Nous avons scrapé 979 bières.


### Étape 3 : on convertit ensuite notre dictionnaire en dataframe

In [10]:
df_beers = pd.DataFrame.from_dict(dico_all_beers, orient='index') #faire une table à partir d'un dictionnaire
df_beers

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Brugse Zot blonde,Brugse Zot blonde,Bière Blonde,Belgique,Brouwerij De Halve Maan,33 cl,"6,0%",23,60,40,20,40,0,"€ 2,69","(3,71)"
Brouwerij 't IJ Natte Dubbel,Brouwerij 't IJ Natte Dubbel,Bière Double,Pays-Bas,Brouwerij 't IJ,33 cl,"6,5%",-,60,60,0,40,0,"€ 2,65","(3,44)"
Bruut Gajes Tripel,Bruut Gajes Tripel,Bière Triple,Pays-Bas,Bruut Bier,33 cl,"8,0%",38,60,60,40,60,0,"€ 2,99","(3,76)"
Baxbier Koud Vuur,Baxbier Koud Vuur,Bière Brune,Pays-Bas,Bax Bier Brouwerij,33 cl,"6,2%",38,80,40,20,60,0,"€ 3,09","(3,37)"
Bacchus Framboos,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38 cl,"5,0%",-,80,60,60,20,0,"€ 3,49","(3,60)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hertog Jan Grand Prestige,Hertog Jan Grand Prestige,Bière Barley Wine,Pays-Bas,NaN,30 cl,"10,0%",25,60,80,0,40,0,"€ 2,39","(3,83)"
Kehrwieder Kreativbrauerei SHIPA Hull Melon,Kehrwieder Kreativbrauerei SHIPA Hull Melon,IPA,Allemagne,NaN,33 cl,"7,5%",65,60,60,0,60,0,"€ 2,99","(3,69)"
Kraftbier Schoon Mèdje,Kraftbier Schoon Mèdje,IPA,Pays-Bas,NaN,33 cl,"6,3%",65,60,40,20,60,0,"€ 2,89","(3,46)"
Grisette Blanche Bio,Grisette Blanche Bio,Bière Blanche,Belgique,St. Feuillien,25 cl,"5,5%",26,60,40,40,0,0,"€ 1,99","(3,52)"


On finalise notre tableau

In [11]:
df_beers.columns = ['beer_name', 'beer_style', 'country', 'brewery_name', 'bottle', 'abv', 'ibu', 'intensite', 'longueur', 'acidite', 'amertume', 'umami', 'price', 'note']
df_beers

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
Brugse Zot blonde,Brugse Zot blonde,Bière Blonde,Belgique,Brouwerij De Halve Maan,33 cl,"6,0%",23,60,40,20,40,0,"€ 2,69","(3,71)"
Brouwerij 't IJ Natte Dubbel,Brouwerij 't IJ Natte Dubbel,Bière Double,Pays-Bas,Brouwerij 't IJ,33 cl,"6,5%",-,60,60,0,40,0,"€ 2,65","(3,44)"
Bruut Gajes Tripel,Bruut Gajes Tripel,Bière Triple,Pays-Bas,Bruut Bier,33 cl,"8,0%",38,60,60,40,60,0,"€ 2,99","(3,76)"
Baxbier Koud Vuur,Baxbier Koud Vuur,Bière Brune,Pays-Bas,Bax Bier Brouwerij,33 cl,"6,2%",38,80,40,20,60,0,"€ 3,09","(3,37)"
Bacchus Framboos,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38 cl,"5,0%",-,80,60,60,20,0,"€ 3,49","(3,60)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hertog Jan Grand Prestige,Hertog Jan Grand Prestige,Bière Barley Wine,Pays-Bas,NaN,30 cl,"10,0%",25,60,80,0,40,0,"€ 2,39","(3,83)"
Kehrwieder Kreativbrauerei SHIPA Hull Melon,Kehrwieder Kreativbrauerei SHIPA Hull Melon,IPA,Allemagne,NaN,33 cl,"7,5%",65,60,60,0,60,0,"€ 2,99","(3,69)"
Kraftbier Schoon Mèdje,Kraftbier Schoon Mèdje,IPA,Pays-Bas,NaN,33 cl,"6,3%",65,60,40,20,60,0,"€ 2,89","(3,46)"
Grisette Blanche Bio,Grisette Blanche Bio,Bière Blanche,Belgique,St. Feuillien,25 cl,"5,5%",26,60,40,40,0,0,"€ 1,99","(3,52)"


### Étape 4 : on nettoie les données pour leur exploitation future 

On convertir les chaines de caractères en `float` pour plus tard traiter les données

In [12]:
for i in range(len(df_beers)):
    
    if type(df_beers['bottle'][i]) != float:
        df_beers['bottle'][i] = float(df_beers['bottle'][i].replace(' cl',''))

    if type(df_beers['abv'][i]) != float:
        df_beers['abv'][i] = float(df_beers['abv'][i].replace('%','').replace(',','.'))
    
    if df_beers['ibu'][i] == '-':
        df_beers['ibu'][i] = float('nan')
    elif type(df_beers['ibu'][i]) == str and len(df_beers['ibu'][i]) > 6:
        df_beers['ibu'][i] = float('nan')
    else:
        df_beers['ibu'][i] = float(df_beers['ibu'][i])

    if type(df_beers['note'][i]) != float:
        df_beers['note'][i] = float(df_beers['note'][i].replace('(','').replace(')','').replace(',','.'))

    if type(df_beers['price'][i]) != float:
        df_beers['price'][i] = float(df_beers['price'][i].replace('€ ','').replace(',','.'))
    
df_beers

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,umami,price,note
Brugse Zot blonde,Brugse Zot blonde,Bière Blonde,Belgique,Brouwerij De Halve Maan,33,6,23,60,40,20,40,0,2.69,3.71
Brouwerij 't IJ Natte Dubbel,Brouwerij 't IJ Natte Dubbel,Bière Double,Pays-Bas,Brouwerij 't IJ,33,6.5,NaN,60,60,0,40,0,2.65,3.44
Bruut Gajes Tripel,Bruut Gajes Tripel,Bière Triple,Pays-Bas,Bruut Bier,33,8,38,60,60,40,60,0,2.99,3.76
Baxbier Koud Vuur,Baxbier Koud Vuur,Bière Brune,Pays-Bas,Bax Bier Brouwerij,33,6.2,38,80,40,20,60,0,3.09,3.37
Bacchus Framboos,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38,5,NaN,80,60,60,20,0,3.49,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hertog Jan Grand Prestige,Hertog Jan Grand Prestige,Bière Barley Wine,Pays-Bas,NaN,30,10,25,60,80,0,40,0,2.39,3.83
Kehrwieder Kreativbrauerei SHIPA Hull Melon,Kehrwieder Kreativbrauerei SHIPA Hull Melon,IPA,Allemagne,NaN,33,7.5,65,60,60,0,60,0,2.99,3.69
Kraftbier Schoon Mèdje,Kraftbier Schoon Mèdje,IPA,Pays-Bas,NaN,33,6.3,65,60,40,20,60,0,2.89,3.46
Grisette Blanche Bio,Grisette Blanche Bio,Bière Blanche,Belgique,St. Feuillien,25,5.5,26,60,40,40,0,0,1.99,3.52


Exportation du tableau en tant que fichier csv pour ensuite être traité

In [13]:
df_beers.to_csv('data_beerwulf')

**Les étapes suivantes : data cleaning**, par exemple :
- Mettre le prix au litre pour uniformiser.
- Faire le lien avec les autres bases de données existantes